In [ ]:
from Bio import Entrez
import pandas as pd
import os
import time
import json
from datetime import datetime
import xml.etree.ElementTree as ET

In [ ]:
Entrez.email = "olandechris@gmail.com"

In [ ]:
def search_pubmed(query, max_results = 1, date_from = None, date_to = None, sort_order = "relevance", publication_types = None):
    
    search_term = query

    # Add date filters
    if date_from or date_to:
        if date_from and date_to:
            search_term += f' AND {date_from}[PDAT]:{date_to}[PDAT]'
        elif date_from:
            search_term += f' AND {date_from}[PDAT]:3000[PDAT]'
        elif date_to:
            search_term += f' AND 1900[PDAT]:{date_to}[PDAT]'

    # Add publication type filters
    if publication_types:
        pub_filter = ' OR '.join([f'"{pt}"[Publication Type]' for pt in publication_types])
        search_term += f' AND ({pub_filter})'
    
    print(f"Searching PubMed with query: {search_term}")

    try: 
        # Perform the search
        handle = Entrez.esearch(
            db = "pubmed",
            term=search_term,
            retmax=max_results,
            sort=sort_order
        )

        search_results = Entrez.read(handle)

        handle.close()

        pmids = search_results["IdList"]
        count = int(search_results["Count"])
        
        print(f"Found {count} total articles, retrieving {len(pmids)} IDs")
        return pmids

    except Exception as e:
        print(f"Search error: {e}")
        return []

        

In [ ]:
search_pubmed("Kurt Cobain")

In [ ]:
https://pubmed.ncbi.nlm.nih.gov/7988166

In [ ]:
def parse_article(summary, record):
    """Parse article summary and record into structured data"""
    try:
        # Basic info from summary
        pmid = str(summary.get('Id', ''))
        title = summary.get('Title', '').strip()
        
        # Journal info
        journal = summary.get('Source', '')
        pub_date = summary.get('PubDate', '')
        
        # Authors from summary
        authors_list = summary.get('AuthorList', [])
        authors = '; '.join([author for author in authors_list]) if authors_list else ''
        
        # Extract more details from full record
        article = record['MedlineCitation']['Article']
        
        # Abstract
        abstract = ''
        if 'Abstract' in article:
            abstract_texts = []
            if 'AbstractText' in article['Abstract']:
                for abs_text in article['Abstract']['AbstractText']:
                    if isinstance(abs_text, str):
                        abstract_texts.append(abs_text)
                    else:
                        # Handle structured abstracts with labels
                        abstract_texts.append(str(abs_text))
            abstract = ' '.join(abstract_texts)
        
        # Publication details
        journal_info = article.get('Journal', {})
        journal_title = journal_info.get('Title', journal)
        
        # Volume and issue
        journal_issue = journal_info.get('JournalIssue', {})
        volume = journal_issue.get('Volume', '')
        issue = journal_issue.get('Issue', '')
        
        # Publication date details
        pub_date_info = journal_issue.get('PubDate', {})
        year = pub_date_info.get('Year', '')
        month = pub_date_info.get('Month', '')
        day = pub_date_info.get('Day', '')
        
        # DOI and other identifiers
        doi = ''
        pmc_id = ''
        
        if 'ELocationID' in article:
            for eloc in article['ELocationID']:
                if eloc.attributes.get('EIdType') == 'doi':
                    doi = str(eloc)
                elif eloc.attributes.get('EIdType') == 'pmc':
                    pmc_id = str(eloc)
        
        # Keywords/MeSH terms
        mesh_terms = []
        if 'MeshHeadingList' in record['MedlineCitation']:
            for mesh in record['MedlineCitation']['MeshHeadingList']:
                descriptor = mesh['DescriptorName']
                mesh_terms.append(str(descriptor))
        
        # Publication types
        pub_types = []
        if 'PublicationTypeList' in article:
            pub_types = [str(pt) for pt in article['PublicationTypeList']]
        
        return {
            'pmid': pmid,
            'title': title,
            'abstract': abstract,
            'authors': authors,
            'journal': journal_title,
            'volume': volume,
            'issue': issue,
            'year': year,
            'month': month,
            'day': day,
            'pub_date': pub_date,
            'doi': doi,
            'pmc_id': pmc_id,
            'mesh_terms': '; '.join(mesh_terms),
            'publication_types': '; '.join(pub_types),
            'pubmed_url': f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/",
            'doi_url': f"https://doi.org/{doi}" if doi else ''
        }
        
    except Exception as e:
        print(f"Error parsing article {pmid}: {e}")
        return None

In [ ]:
def fetch_article_details(pmids, batch_size = 100):
    articles = []

    # Process in batches
    for i in range(0, len(pmids), batch_size):
        batch_pmids = pmids[i:i + batch_size]

        print(f"Fetching batch {i//batch_size + 1}/{(len(pmids)-1)//batch_size + 1} "
                  f"({len(batch_pmids)} articles)...")

        try:
            # Fetch article summaries first
            handle = Entrez.esummary(db = "pubmed", id = ",".join(batch_pmids))
            summaries = Entrez.read(handle)
            handle.close()

            # Fetch full abstract records
            handle = Entrez.efetch(
                db = "pubmed",
                id = ",".join(batch_pmids),
                rettype = "medline",
                retmode = "xml"
            )

            records = Entrez.read(handle)
            handle.close()

            # Parse articles
            for summary, record in zip(summaries, records['PubmedArticle']):
                article_data = parse_article(summary, record)
                if article_data:
                    articles.append(article_data)
            
            # Simple rate limiter
            #TODO: Implement the exponential backoff if need be
            time.sleep(0.34) # 3 requests per second

        except Exception as e:
            print(f"Error fetching batch {i//batch_size + 1}: {e}")
            continue
    
    return articles

In [ ]:
fetch_article_details(['7988166'])

In [ ]:
def advanced_search(**kwargs):
    """
    Perform advanced search with multiple parameters
    
    Available parameters:
    - query: Main search terms
    - author: Author name
    - journal: Journal name
    - mesh_terms: MeSH terms list
    - title_words: Words that must appear in title
    - abstract_words: Words that must appear in abstract
    - date_from/date_to: Date range
    - publication_types: List of publication types
    - languages: List of languages
    - max_results: Maximum results
    """

    search_parts = []
    # Main query
    if "query" in kwargs:
        search_parts.append(kwargs["query"])

    # Author
    if "author" in kwargs:
        search_parts.append(f'"{kwargs["author"]}"[Author]')
    
    # Journal
    if "journal" in kwargs:
        search_parts.append(f'"{kwargs["journal"]}"[Journal]')

    # Mesh terms
    if "mesh_terms" in kwargs:
        mesh_queries = [f'"{term}"[MeSH Terms]' for term in kwargs['mesh_terms']]
        search_parts.append(f'({" OR ".join(mesh_queries)})')

    # Title words
    if 'title_words' in kwargs:
        title_queries = [f'"{word}"[Title]' for word in kwargs['title_words']]
        search_parts.append(f'({" AND ".join(title_queries)})')
    
    # Abstract words
    if 'abstract_words' in kwargs:
        abstract_queries = [f'"{word}"[Abstract]' for word in kwargs['abstract_words']]
        search_parts.append(f'({" AND ".join(abstract_queries)})')
    
    # Languages
    if 'languages' in kwargs:
        lang_queries = [f'"{lang}"[Language]' for lang in kwargs['languages']]
        search_parts.append(f'({" OR ".join(lang_queries)})')
    
    # Combine all parts
    full_query = ' AND '.join(search_parts)

    return search_pubmed(
            query=full_query,
            max_results=kwargs.get('max_results', 4),
            date_from=kwargs.get('date_from'),
            date_to=kwargs.get('date_to'),
            publication_types=kwargs.get('publication_types')
        )

In [ ]:
advanced_search(query = "Kurt Cobain")

In [ ]:
articles = fetch_article_details(['7988166', '8897665', '26445123', '16179336'])

In [ ]:
def get_article_statistics(articles):
        """Generate basic statistics about downloaded articles"""
        if not articles:
            return {}
        
        df = pd.DataFrame(articles)
        
        stats = {
            'total_articles': len(articles),
            'articles_with_abstracts': len(df[df['abstract'].str.len() > 0]),
            'date_range': {
                'earliest': df['year'].min(),
                'latest': df['year'].max()
            },
            'top_journals': df['journal'].value_counts().head(10).to_dict(),
            'publication_types': df['publication_types'].value_counts().head(10).to_dict(),
            'articles_per_year': df['year'].value_counts().sort_index().to_dict()
        }
        
        return stats

In [ ]:
get_article_statistics(articles)

In [ ]:
def save_to_csv(articles, filename):
    """Save articles to CSV using pandas"""
    if not articles:
        print("No articles to save")
        return
        
    df = pd.DataFrame(articles)
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"Saved {len(articles)} articles to {filename}")

def save_to_excel(articles, filename):
    """Save articles to Excel file"""
    if not articles:
        print("No articles to save")
        return
    
    df = pd.DataFrame(articles)
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"Saved {len(articles)} articles to {filename}")

def save_to_json(articles, filename):
    """Save articles to JSON file"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(articles, f, indent=2, ensure_ascii=False)
    print(f"Saved {len(articles)} articles to {filename}")

In [ ]:
save_to_csv(articles, "Cobain.csv")

# The mismatch count returned by the vector store
The vector store says that the number of documents in the graph is 2451, while we uploaded 4833 documents, a suggestion that we might be duplicating things

In [ ]:
import json
from langchain_core.documents import Document

documents_path = "output/pmc_chunks/pmc_semantic_chunks.json"

with open(documents_path, "r", encoding = "utf-8") as f:
    data = json.load(f)

loaded_documents_count = len(data.get("documents", []))
print(f"Number of documents (chunks) loaded from JSON: {loaded_documents_count}")

documents = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in data.get("documents", [])]
print(f"Length of the 'documents' list after list comprehension: {len(documents)}")

In [ ]:

from collections import defaultdict, Counter

# Group by pmid to see the pmid_seq_num pattern
pmid_groups = defaultdict(list)
doc_ids = []

for doc in documents:
    pmid = doc.metadata.get("pmid", " ")
    seq_num = doc.metadata.get("seq_num", " ")

    doc_id = f"{pmid}_{seq_num}"

    pmid_groups[pmid].append(seq_num)
    doc_ids.append(doc_id)

In [ ]:
# Check for duplicates
id_counts = Counter(doc_ids)
duplicates = {doc_id: count for doc_id, count in id_counts.items() if count > 1}
print(f"Total documents: {len(documents)}")
print(f"Unique PMIDs: {len(pmid_groups)}")
print(f"Duplicate pmid_seq_num combinations: {len(duplicates)}")

In [ ]:
pmid_groups = defaultdict(list)
for i, doc in enumerate(documents):
    pmid = doc.metadata.get("pmid", " ")
    pmid_groups[pmid].append((i, doc))

fixed_count = 0
for pmid, doc_list in pmid_groups.items():
    if len(doc_list) > 1: # Process pmids with multiple chunks
        print(f"Fixing PMID {pmid}: {len(doc_list)} chunks")

        # Sort by original order
        doc_list.sort(key = lambda x: x[0])

        # Reassign seq_num: 0, 1, 2, 3...
        for new_seq, (original_idx, doc) in enumerate(doc_list):
            old_seq = doc.metadata.get('seq_num', '')
            doc.metadata['seq_num'] = new_seq
            print(f"  Changed seq_num from {old_seq} to {new_seq}")
            fixed_count += 1

print(f"Fixed {fixed_count} documents total")
        

In [ ]:

# Group documents by pmid
pmid_groups = defaultdict(list)
for i, doc in enumerate(documents):
    pmid = doc.metadata.get('pmid', '')
    pmid_groups[pmid].append((i, doc))

# Fix seq_num for each pmid group
fixed_count = 0
for pmid, doc_list in pmid_groups.items():
    if len(doc_list) > 1:  # Only process PMIDs with multiple chunks
        print(f"Fixing PMID {pmid}: {len(doc_list)} chunks")
        
        # Sort by original order
        doc_list.sort(key=lambda x: x[0])
        
        # Reassign seq_num: 0, 1, 2, 3...
        for new_seq, (original_idx, doc) in enumerate(doc_list):
            old_seq = doc.metadata.get('seq_num', '')
            doc.metadata['seq_num'] = new_seq
            print(f"  Changed seq_num from {old_seq} to {new_seq}")
            fixed_count += 1

print(f"Fixed {fixed_count} documents total")


In [ ]:
from langchain.globals import set_llm_cache
semantic_cache = SemanticCache(
    database_path=".my_semantic_cache.db",
    faiss_index_path="./my_semantic_faiss_index",
    similarity_threshold=0.5,
    max_cache_size=1000,        # FAISS index size limit
    memory_cache_size=100,      # In-memory cache size
    batch_size=10,              # Batch processing size
    enable_quantization=True    # Enable for large datasets
)

set_llm_cache(semantic_cache)


In [ ]:
semantic_cache.clear_cache()

In [ ]:
embedding = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-MiniLM-L6-v2")
documents = [
        "Python is a high-level programming language known for its simplicity",
        "Java is an object-oriented programming language used for enterprise applications", 
        "Machine learning algorithms automatically learn patterns from data",
        "Deep learning is a subset of machine learning using neural networks",
        "FAISS enables efficient similarity search and clustering of dense vectors",
        "Elasticsearch is a distributed search and analytics engine",
        "BM25 is a probabilistic ranking function used in information retrieval",
        "TF-IDF measures term importance in document collections",
        "Vector databases are optimized for storing high-dimensional embeddings",
        "SQL databases use structured query language for data management",
        "Natural language processing enables computers to understand human language",
        "Computer vision algorithms process and analyze visual data",
        "Recommendation systems suggest relevant items to users",
        "Time series analysis forecasts future values from historical data",
        "Graph databases model relationships between entities"
    ]
for doc in documents:
    print(embedding.embed_query(doc))

In [ ]:
import asyncio
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_cohere import CohereEmbeddings
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
from typing import List, Dict

class RetrievalEvaluator:
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-MiniLM-L6-v2")
        
    async def setup_retrievers(self, documents: List[str]) -> Dict:
        """Setup FAISS, BM25, and RRF retrievers"""
        docs = [Document(page_content=doc) for doc in documents]
        
        # FAISS retriever
        faiss_store = await FAISS.afrom_documents(docs, self.embeddings)
        faiss_retriever = faiss_store.as_retriever(search_kwargs={"k": 5})
        
        # BM25 retriever
        bm25_retriever = BM25Retriever.from_documents(docs)
        bm25_retriever.k = 5
        
        # RRF (Ensemble) retriever combining FAISS + BM25
        rrf_retriever = EnsembleRetriever(
            retrievers=[faiss_retriever, bm25_retriever],
            weights=[0.5, 0.5]
        )
        
        return {
            "faiss": faiss_retriever,
            "bm25": bm25_retriever, 
            "rrf": rrf_retriever
        }
    
    async def evaluate_retrieval(self, retrievers: Dict, queries: List[str], 
                               ground_truth: List[List[int]]) -> Dict:
        """Evaluate retrievers using MRR, Recall@K, and Precision@K"""
        results = {}
        
        for name, retriever in retrievers.items():
            mrr_scores = []
            recall_scores = []
            precision_scores = []
            
            for query, relevant_docs in zip(queries, ground_truth):
                retrieved = await retriever.ainvoke(query)
                retrieved_indices = [int(doc.metadata.get('index', -1)) 
                                   for doc in retrieved if 'index' in doc.metadata]
                
                # MRR calculation
                mrr = 0
                for i, doc_idx in enumerate(retrieved_indices):
                    if doc_idx in relevant_docs:
                        mrr = 1 / (i + 1)
                        break
                mrr_scores.append(mrr)
                
                # Recall@K and Precision@K
                relevant_retrieved = set(retrieved_indices) & set(relevant_docs)
                recall = len(relevant_retrieved) / len(relevant_docs) if relevant_docs else 0
                precision = len(relevant_retrieved) / len(retrieved_indices) if retrieved_indices else 0
                
                recall_scores.append(recall)
                precision_scores.append(precision)
            
            results[name] = {
                "MRR": np.mean(mrr_scores),
                "Recall@5": np.mean(recall_scores),
                "Precision@5": np.mean(precision_scores)
            }
        
        return results

# Usage example
async def main():
    # More realistic dataset with noise
    documents = [
        "Python is a high-level programming language known for its simplicity",
        "Java is an object-oriented programming language used for enterprise applications", 
        "Machine learning algorithms automatically learn patterns from data",
        "Deep learning is a subset of machine learning using neural networks",
        "FAISS enables efficient similarity search and clustering of dense vectors",
        "Elasticsearch is a distributed search and analytics engine",
        "BM25 is a probabilistic ranking function used in information retrieval",
        "TF-IDF measures term importance in document collections",
        "Vector databases are optimized for storing high-dimensional embeddings",
        "SQL databases use structured query language for data management",
        "Natural language processing enables computers to understand human language",
        "Computer vision algorithms process and analyze visual data",
        "Recommendation systems suggest relevant items to users",
        "Time series analysis forecasts future values from historical data",
        "Graph databases model relationships between entities"
    ]
    
    # Add document indices to metadata
    docs_with_metadata = []
    for i, doc in enumerate(documents):
        docs_with_metadata.append(Document(page_content=doc, metadata={"index": i}))
    
    queries = [
        "How to program in Python?",
        "What are machine learning techniques?", 
        "Vector similarity search methods",
        "Database query languages",
        "Text analysis and processing"
    ]
    
    # Ground truth: multiple relevant docs per query
    ground_truth = [
        [0, 1],      # Python query -> Python + Java docs
        [2, 3],      # ML query -> ML + Deep learning
        [4, 8],      # Vector query -> FAISS + Vector DBs  
        [5, 9],      # Database query -> Elasticsearch + SQL
        [7, 10, 11]  # Text analysis -> TF-IDF + NLP + CV
    ]
    
    evaluator = RetrievalEvaluator()
    
    # Setup retrievers with metadata-enriched documents
    retrievers = {}
    docs = docs_with_metadata
    
    # FAISS
    faiss_store = await FAISS.afrom_documents(docs, evaluator.embeddings)
    retrievers["faiss"] = faiss_store.as_retriever(search_kwargs={"k": 5})
    
    # BM25  
    retrievers["bm25"] = BM25Retriever.from_documents(docs)
    retrievers["bm25"].k = 5
    
    # RRF
    retrievers["rrf"] = EnsembleRetriever(
        retrievers=[retrievers["faiss"], retrievers["bm25"]],
        weights=[0.5, 0.5]
    )
    
    # Evaluate
    results = await evaluator.evaluate_retrieval(retrievers, queries, ground_truth)
    
    # Print results
    for method, metrics in results.items():
        print(f"\n{method.upper()} Results:")
        for metric, score in metrics.items():
            print(f"  {metric}: {score:.3f}")

if __name__ == "__main__":
    await main()

In [ ]:
import pandas as pd

pd.read_csv("retrieval_evaluation_results.csv")